In [398]:
import pandas as pd
import hashlib
import re

### Load

In [399]:
filename= "../data/export_15_03_2025_10_47_36.xls"
df = pd.read_excel(filename, skiprows=2)
df.head()

,Date operation,Categorie operation,Sous Categorie operation,Libelle operation,Montant operation,Pointage operation,Commentaire operation
0,14-03-2025,A Categoriser,Virement interne,VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT ...,-64878.00,NaN,NaN
1,14-03-2025,Vie Quotidienne,"Achat multimedia, hightech","PAIEMENT CB ADOBE DU 13/03/25 A SAGGART, DUBL ...",-11.99,NaN,NaN
2,14-03-2025,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB BISTRO DU FAUBO (FRANCE) DU 12/03/...,-7.00,NaN,NaN
3,14-03-2025,Vie Quotidienne,"Achat multimedia, hightech",PAIEMENT CB APPLE (IRLANDE) DU 13/03/25 - CART...,-3.99,NaN,NaN
4,13-03-2025,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB CARAVANA (FRANCE) DU 12/03/25 - CA...,-13.90,NaN,NaN


### Processing

Add extraction date timestamp in a column

In [400]:
_, day, month, year, hour, minute, second = filename.replace(".xls", "").split("_")
extracted_date = f"{year}-{month}-{day} {hour}:{minute}:{second}"
df["Date Extraction"] = extracted_date
df.head()

,Date operation,Categorie operation,Sous Categorie operation,Libelle operation,Montant operation,Pointage operation,Commentaire operation,Date Extraction
0,14-03-2025,A Categoriser,Virement interne,VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT ...,-64878.00,NaN,NaN,2025-03-15 10:47:36
1,14-03-2025,Vie Quotidienne,"Achat multimedia, hightech","PAIEMENT CB ADOBE DU 13/03/25 A SAGGART, DUBL ...",-11.99,NaN,NaN,2025-03-15 10:47:36
2,14-03-2025,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB BISTRO DU FAUBO (FRANCE) DU 12/03/...,-7.00,NaN,NaN,2025-03-15 10:47:36
3,14-03-2025,Vie Quotidienne,"Achat multimedia, hightech",PAIEMENT CB APPLE (IRLANDE) DU 13/03/25 - CART...,-3.99,NaN,NaN,2025-03-15 10:47:36
4,13-03-2025,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB CARAVANA (FRANCE) DU 12/03/25 - CA...,-13.90,NaN,NaN,2025-03-15 10:47:36


Convert types

In [401]:
df["Date operation"] = pd.to_datetime(df["Date operation"])
df['Date Extraction'] = pd.to_datetime(df['Date Extraction'])

/var/folders/64/lbsssmms29n9r2sh2cm9dxv00000gn/T/ipykernel_58485/2130239904.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["Date operation"] = pd.to_datetime(df["Date operation"])


Add id

In [402]:
df['id'] = df['Date operation'].astype(str) + "_" + df['Categorie operation'].astype(str) + "_" + df['Sous Categorie operation'].astype(str) + "_" + df['Libelle operation'].astype(str) + "_" + df['Montant operation'].astype(str)
df['id'] = df['id'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())
col = df.pop('id')
df.insert(0, 'id', col)

Add day/month/year

In [403]:
df["Jour"] = df["Date operation"].dt.day
df["Mois"] = df["Date operation"].dt.month
df["Année"] = df["Date operation"].dt.year
df["Mois-Annee"] = df["Date operation"].dt.strftime("%m-%Y")

Remove unused columns

In [404]:
df.drop(columns=['Pointage operation'], inplace=True)
df.drop(columns=['Commentaire operation'], inplace=True)

In [405]:
df['Libelle operation'].tolist()

['VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT TAMADON',
 'PAIEMENT CB ADOBE DU 13/03/25 A SAGGART, DUBL - CARTE*5333',
 'PAIEMENT CB BISTRO DU FAUBO (FRANCE) DU 12/03/25 - CARTE*5333',
 'PAIEMENT CB APPLE (IRLANDE) DU 13/03/25 - CARTE*5333',
 'PAIEMENT CB CARAVANA (FRANCE) DU 12/03/25 - CARTE*5333',
 'PAIEMENT CB HPY*LE COQ SPOR DU 08/03/25 A ROMILLY-SUR-S - CARTE*5333',
 'PAIEMENT CB NEXANS FRANCE S (FRANCE) DU 10/03/25 - CARTE*5333',
 'PAIEMENT CB OPENAI *CHATGPT (ETATS-UNIS) DU 06/03/25 - CARTE*5333',
 'PRELEVEMENT CHUBB EUROPEAN GROUP SE CHUBB EUROPEAN GROUP SE DU 10/03/25 - EMETTEUR : FR39ZZZ495463 MDT - MOTIF : 148699415 - REF : CIGSP70D 26',
 'PAIEMENT CB SUSHI HOUSE (FRANCE) DU 07/03/25 - CARTE*5333',
 'PAIEMENT CB BOULANGERIE MAR (FRANCE) DU 07/03/25 - CARTE*5333',
 'PAIEMENT CB POISSONNIERE (FRANCE) DU 06/03/25 - CARTE*5333',
 'PAIEMENT CB MEDIUM MONTHLY (ETATS-UNIS) DU 06/03/25 - CARTE*5333',
 'PAIEMENT CB APPLE (IRLANDE) DU 06/03/25 - CARTE*5333',
 'PAIEMENT CB CARAVANA (

Clean description

In [406]:
def clean_transaction_description(df):
    """
    Nettoie les descriptions des transactions en supprimant les dates (format DU JJ/MM/AA)
    et les références de carte (format - CARTE*XXXX).

    Args:
        df (pd.DataFrame): DataFrame contenant la colonne à nettoyer.
        column_name (str): Nom de la colonne à modifier.

    Returns:
        pd.DataFrame: DataFrame avec la colonne nettoyée.
    """
    df = df.copy()  # Pour éviter de modifier l'original
    df['Libelle clean'] = df['Libelle operation']

    # Expression régulière pour supprimer "DU JJ/MM/YY" et "- CARTE*XXXX"
    df['Libelle clean'] = df['Libelle clean'].str.replace(r"DU \d{2}/\d{2}/\d{2}", "", regex=True)
    df['Libelle clean'] = df['Libelle clean'].str.replace(r"- CARTE\*\d+", "", regex=True)

    # Nettoyage des espaces inutiles
    df['Libelle clean'] = df['Libelle clean'].str.replace(r"\s+", " ", regex=True).str.strip()

    return df

df = clean_transaction_description(df)
df.head()

,id,Date operation,Categorie operation,Sous Categorie operation,Libelle operation,Montant operation,Date Extraction,Jour,Mois,Année,Mois-Annee,Libelle clean
0,77f8493ca24f02dea17cb67965c5258a73fc4fde75e7b2...,2025-03-14,A Categoriser,Virement interne,VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT ...,-64878.00,2025-03-15 10:47:36,14,3,2025,03-2025,VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT ...
1,942d2e6eae9a231cc72985bdc08804aada0b9d36d372a7...,2025-03-14,Vie Quotidienne,"Achat multimedia, hightech","PAIEMENT CB ADOBE DU 13/03/25 A SAGGART, DUBL ...",-11.99,2025-03-15 10:47:36,14,3,2025,03-2025,"PAIEMENT CB ADOBE A SAGGART, DUBL"
2,941bb1735f80c2267aac1c36206a2eae90058e2f740d9a...,2025-03-14,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB BISTRO DU FAUBO (FRANCE) DU 12/03/...,-7.00,2025-03-15 10:47:36,14,3,2025,03-2025,PAIEMENT CB BISTRO DU FAUBO (FRANCE)
3,a09fdabc5afb7f14f4275c530543500f1e1294e929fd2f...,2025-03-14,Vie Quotidienne,"Achat multimedia, hightech",PAIEMENT CB APPLE (IRLANDE) DU 13/03/25 - CART...,-3.99,2025-03-15 10:47:36,14,3,2025,03-2025,PAIEMENT CB APPLE (IRLANDE)
4,4b45642f04eeb3b3f50599b08b28c403fe6cf684a80ae5...,2025-03-13,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB CARAVANA (FRANCE) DU 12/03/25 - CA...,-13.90,2025-03-15 10:47:36,13,3,2025,03-2025,PAIEMENT CB CARAVANA (FRANCE)


Detect single paiements in month

In [407]:
# def detect_single_month_payment_flag(df):
#     """
#     Ajoute une colonne 'is_single_month_payment' au DataFrame,
#     indiquant si une opération apparaît une seule fois dans le mois.

#     Args:
#         df (pd.DataFrame): DataFrame contenant une colonne 'Date operation' et 'Libelle operation'.

#     Returns:
#         pd.DataFrame: DataFrame avec la nouvelle colonne 'is_single_month_payment'.
#     """

#     # Compter les occurrences de chaque opération dans un mois donné
#     df["operation_count"] = df.groupby(["Mois-Annee", "Libelle clean"])["Libelle clean"].transform("count")

#     # Marquer les opérations uniques dans le mois
#     df["is_single_month_payment"] = df["operation_count"] == 1

#     # Supprimer la colonne temporaire
#     df.drop(columns=["operation_count"], inplace=True)

#     return df

# df = detect_single_month_payment_flag(df)
# df.head()

Detect subscription

In [419]:
def detect_subscriptions_v0(df):
    """
    Ajoute une colonne 'is_subscription' au dataframe en détectant les paiements récurrents 
    sur au moins 3 mois consécutifs avec exactement le même montant.

    :param df: DataFrame contenant au moins les colonnes 'Date operation' et 'Montant operation'
    :return: DataFrame avec la colonne 'is_subscription' ajoutée
    """

    # Comptage du nombre d'occurrences par mois et par montant
    monthly_counts = df.groupby(['Montant operation', 'Mois-Annee'])['Libelle clean'].count().reset_index()

    # Filtrage des montants qui apparaissent exactement 1 fois par mois
    valid_monthly = monthly_counts[monthly_counts['Libelle clean'] == 1]

    # Comptage du nombre de mois distincts pour chaque montant
    subscription_candidates = valid_monthly.groupby('Montant operation')['Mois-Annee'].nunique().reset_index()

    # Identifier les montants correspondant à un abonnement (au moins 3 mois consécutifs)
    subscription_candidates = subscription_candidates[subscription_candidates['Mois-Annee'] >= 3]

    # Ajouter la colonne is_subscription au dataframe original
    df['is_subscription'] = df['Montant operation'].isin(subscription_candidates['Montant operation'])

    return df

df = detect_subscriptions(df)
df[df.is_subscription == True].sort_values('Libelle operation', ascending=False).head(50)

# .sort_values('Montant operation', ascending=False)

,id,Date operation,Categorie operation,Sous Categorie operation,Libelle operation,Montant operation,Date Extraction,Jour,Mois,Année,Mois-Annee,Libelle clean,is_single_value,is_subscription
439,e2d144ec587644090e1d8a03a693b5e65a0575267bab9e...,2024-08-12,A Categoriser,Virement emis,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,-3.00,2025-03-15 10:47:36,12,8,2024,08-2024,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,False,True
263,c42324d4f5e0811e6f3f3593bf227bee6056c1dae92cf9...,2024-11-12,A Categoriser,Virement emis,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,-3.00,2025-03-15 10:47:36,12,11,2024,11-2024,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,False,True
546,7c22a0d7df7fc86e06c0393dce4776e226c46c95a97bad...,2024-06-11,A Categoriser,Virement emis,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,-3.00,2025-03-15 10:47:36,11,6,2024,06-2024,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,True,True
323,0162ed81ab07484a3f9881deaa327b3a54095361d785ea...,2024-10-11,A Categoriser,Virement emis,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,-3.00,2025-03-15 10:47:36,11,10,2024,10-2024,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,False,True
652,21815d51be71b7b2cfe896e614de5a9f825b0c3f14d5d7...,2024-04-11,A Categoriser,Virement emis,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,-3.00,2025-03-15 10:47:36,11,4,2024,04-2024,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,True,True
498,58c9c67e73bb266b7bc8de4d3a49bd978a1a51f932244f...,2024-07-11,A Categoriser,Virement emis,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,-3.00,2025-03-15 10:47:36,11,7,2024,07-2024,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,True,True
369,d85440b6b50f68044bde82456cc5271af4c424917bf3ba...,2024-09-11,A Categoriser,Virement emis,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,-3.00,2025-03-15 10:47:36,11,9,2024,09-2024,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,True,True
187,24f40333a8e26b6551f0577be94b642cb74168468c4564...,2024-12-11,A Categoriser,Virement emis,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,-3.00,2025-03-15 10:47:36,11,12,2024,12-2024,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,False,True
706,c46e781abbc4420af6a9549a6950b8f8a23d856ff155b4...,2024-03-11,A Categoriser,Virement emis,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,-3.00,2025-03-15 10:47:36,11,3,2024,03-2024,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,True,True
608,4792379d488eddf1b6aed24bc1716f3004b7f455d33af9...,2024-05-13,A Categoriser,Virement emis,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,-3.00,2025-03-15 10:47:36,13,5,2024,05-2024,VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE...,True,True


In [420]:
pd.DataFrame(df['Libelle clean'].value_counts()).head(50)

,count
Libelle clean,
PAIEMENT CB APPLE (IRLANDE),52
PAIEMENT CB LYOVEL SAS (FRANCE),14
COMMISSIONS COTISATION A UNE OFFRE GROUPEE DE SERVICES ESPRIT LIBRE,13
PAIEMENT CB AWS EMEA (LUXEMBOURG),13
PAIEMENT CB AMAZON A PAYLI2441535,12
PAIEMENT CB BALTAIAN ET COM (FRANCE),12
PAIEMENT CB WTA TV (ROYAUME-UNI),11
PAIEMENT CB FRANPRIX (FRANCE),10
VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE MOULA AUGMENTE,10


In [409]:
pd.DataFrame(df[df.is_subscription == True]['Libelle clean'].value_counts()).head(50)

,count
Libelle clean,
PAIEMENT CB APPLE (IRLANDE),50
COMMISSIONS COTISATION A UNE OFFRE GROUPEE DE SERVICES ESPRIT LIBRE,13
PAIEMENT CB WTA TV (ROYAUME-UNI),11
VIREMENT PERMANENT - MOTIF : SPOTIFY LOLO : ZE MOULA AUGMENTE,10
"PAIEMENT CB ADOBE A SAGGART, DUBL",8
PAIEMENT CB CLASSPASS* MONT (PAYS-BAS),8
PAIEMENT CB NAVIGO (FRANCE),7
PAIEMENT CB CANAL PLUS A ISSY-LES-MOULINEAUX,7
COMMISSIONS SERVICE AFFINITE,6


In [416]:
df[(df['Libelle operation'].str.contains(r'cantine', case=False, regex=True))]

,id,Date operation,Categorie operation,Sous Categorie operation,Libelle operation,Montant operation,Date Extraction,Jour,Mois,Année,Mois-Annee,Libelle clean,is_single_value,is_subscription
35,cb2cba6321e83d7f4e5313dc526a3ead399f45317e9637...,2025-02-24,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB MA CANTINE CORE (FRANCE) DU 20/02/...,-11.0,2025-03-15 10:47:36,24,2,2025,02-2025,PAIEMENT CB MA CANTINE CORE (FRANCE),False,True
37,29213fa1cc72481d94362b5231d513af16eb0b16a1bae7...,2025-02-21,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB MA CANTINE CORE (FRANCE) DU 19/02/...,-11.0,2025-03-15 10:47:36,21,2,2025,02-2025,PAIEMENT CB MA CANTINE CORE (FRANCE),False,True


In [410]:
df[(df.is_subscription == True) & (df['Libelle operation'].str.contains(r'cantine', case=False, regex=True))]

,id,Date operation,Categorie operation,Sous Categorie operation,Libelle operation,Montant operation,Date Extraction,Jour,Mois,Année,Mois-Annee,Libelle clean,is_single_value,is_subscription
35,cb2cba6321e83d7f4e5313dc526a3ead399f45317e9637...,2025-02-24,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB MA CANTINE CORE (FRANCE) DU 20/02/...,-11.0,2025-03-15 10:47:36,24,2,2025,02-2025,PAIEMENT CB MA CANTINE CORE (FRANCE),False,True
37,29213fa1cc72481d94362b5231d513af16eb0b16a1bae7...,2025-02-21,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB MA CANTINE CORE (FRANCE) DU 19/02/...,-11.0,2025-03-15 10:47:36,21,2,2025,02-2025,PAIEMENT CB MA CANTINE CORE (FRANCE),False,True


In [411]:
test = df[(df.is_subscription == True) & (df['Libelle operation'].str.contains(r'cantine', case=False, regex=True))]
monthly_counts = test.groupby(['Montant operation', 'Mois'])['Libelle operation'].count().reset_index()
valid_monthly = monthly_counts[monthly_counts['Libelle operation'] == 1]
valid_monthly

,Montant operation,Mois,Libelle operation


In [412]:
test['Libelle operation'].tolist()

['PAIEMENT CB MA CANTINE CORE (FRANCE) DU 20/02/25 - CARTE*5333',
 'PAIEMENT CB MA CANTINE CORE (FRANCE) DU 19/02/25 - CARTE*5333']

### Export

In [413]:
df.head()

,id,Date operation,Categorie operation,Sous Categorie operation,Libelle operation,Montant operation,Date Extraction,Jour,Mois,Année,Mois-Annee,Libelle clean,is_single_value,is_subscription
0,77f8493ca24f02dea17cb67965c5258a73fc4fde75e7b2...,2025-03-14,A Categoriser,Virement interne,VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT ...,-64878.00,2025-03-15 10:47:36,14,3,2025,03-2025,VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT ...,True,False
1,942d2e6eae9a231cc72985bdc08804aada0b9d36d372a7...,2025-03-14,Vie Quotidienne,"Achat multimedia, hightech","PAIEMENT CB ADOBE DU 13/03/25 A SAGGART, DUBL ...",-11.99,2025-03-15 10:47:36,14,3,2025,03-2025,"PAIEMENT CB ADOBE A SAGGART, DUBL",True,True
2,941bb1735f80c2267aac1c36206a2eae90058e2f740d9a...,2025-03-14,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB BISTRO DU FAUBO (FRANCE) DU 12/03/...,-7.00,2025-03-15 10:47:36,14,3,2025,03-2025,PAIEMENT CB BISTRO DU FAUBO (FRANCE),True,False
3,a09fdabc5afb7f14f4275c530543500f1e1294e929fd2f...,2025-03-14,Vie Quotidienne,"Achat multimedia, hightech",PAIEMENT CB APPLE (IRLANDE) DU 13/03/25 - CART...,-3.99,2025-03-15 10:47:36,14,3,2025,03-2025,PAIEMENT CB APPLE (IRLANDE),False,True
4,4b45642f04eeb3b3f50599b08b28c403fe6cf684a80ae5...,2025-03-13,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB CARAVANA (FRANCE) DU 12/03/25 - CA...,-13.90,2025-03-15 10:47:36,13,3,2025,03-2025,PAIEMENT CB CARAVANA (FRANCE),False,False


In [414]:
df.dtypes

id                                  object
Date operation              datetime64[ns]
Categorie operation                 object
Sous Categorie operation            object
Libelle operation                   object
Montant operation                  float64
Date Extraction             datetime64[ns]
Jour                                 int32
Mois                                 int32
Année                                int32
Mois-Annee                          object
Libelle clean                       object
is_single_value                       bool
is_subscription                       bool
dtype: object

In [415]:
df.to_excel("../data/bnp_export_15_03.xlsx", index=False)